## 판관비 급여 분리

In [3]:
import pandas as pd
import requests
import json
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import time
time.sleep(0.3)
pd.options.display.float_format = '{:.6f}'.format

In [4]:
sdf = pd.read_csv('data/selling/selling_total.csv')
sdf

,corp_code,corp_name,year,SGAE
0,365387,AJ네트웍스,2019,NaN
1,125080,AK홀딩스,2019,795724941961.000000
2,219097,BGF,2019,101050879822.000000
3,1263022,BGF리테일,2019,859796389576.000000
4,858364,BNK금융지주,2019,NaN
...,...,...,...,...
4381,111421,휴니드테크놀러지스,2024,27557250972.000000
4382,362238,휴비스,2024,84504067956.000000
4383,156488,휴스틸,2024,39800474989.000000
4384,103176,흥국화재,2024,NaN


In [5]:
wdf = pd.read_csv('data/worker/worker_total.csv')
wdf

,corp_code,corp_name,year,sex,full_time,part_time,salary_total,salary_total_man,salary_total_woman,salary_ave,salary_ave_man,salary_ave_woman,ave_work
0,365387,AJ네트웍스,2019,Total,513.000000,18.000000,30112992000.000000,23306579000.000000,6806413000.000000,56709966.101695,67555301.449275,36593618.279570,3.470998
1,125080,AK홀딩스,2019,Total,19.000000,1.000000,1302000000.000000,1103000000.000000,199000000.000000,65100000.000000,68937500.000000,49750000.000000,2.300000
2,219097,BGF,2019,Total,107.000000,6.000000,8246000000.000000,6864000000.000000,1382000000.000000,72973451.327434,82698795.180723,46066666.666667,6.938053
3,1263022,BGF리테일,2019,Total,2010.000000,631.000000,113452000000.000000,92536000000.000000,20916000000.000000,42957970.465733,47723568.849923,29794871.794872,1.900000
4,858364,BNK금융지주,2019,Total,79.000000,17.000000,13196000000.000000,12567000000.000000,629000000.000000,137458333.333333,144448275.862069,69888888.888889,2.273438
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4381,111421,휴니드테크놀러지스,2024,Total,350.000000,9.000000,26354828000.000000,23753408000.000000,2601420000.000000,73411777.158774,76871870.550162,52028400.000000,8.451532
4382,362238,휴비스,2024,Total,646.000000,58.000000,55799000000.000000,49968000000.000000,5831000000.000000,79259943.181818,80593548.387097,69416666.666667,15.529545
4383,156488,휴스틸,2024,Total,563.000000,22.000000,35118000000.000000,33171000000.000000,1947000000.000000,60030769.230769,61314232.902033,44250000.000000,4.225641
4384,103176,흥국화재,2024,Total,864.000000,97.000000,77082382000.000000,52365382000.000000,24717000000.000000,80210595.213319,103899567.460317,54085339.168490,9.524454


In [6]:
wdf = wdf.drop(['corp_code', 'sex'], axis = 1)
wdf

,corp_name,year,full_time,part_time,salary_total,salary_total_man,salary_total_woman,salary_ave,salary_ave_man,salary_ave_woman,ave_work
0,AJ네트웍스,2019,513.000000,18.000000,30112992000.000000,23306579000.000000,6806413000.000000,56709966.101695,67555301.449275,36593618.279570,3.470998
1,AK홀딩스,2019,19.000000,1.000000,1302000000.000000,1103000000.000000,199000000.000000,65100000.000000,68937500.000000,49750000.000000,2.300000
2,BGF,2019,107.000000,6.000000,8246000000.000000,6864000000.000000,1382000000.000000,72973451.327434,82698795.180723,46066666.666667,6.938053
3,BGF리테일,2019,2010.000000,631.000000,113452000000.000000,92536000000.000000,20916000000.000000,42957970.465733,47723568.849923,29794871.794872,1.900000
4,BNK금융지주,2019,79.000000,17.000000,13196000000.000000,12567000000.000000,629000000.000000,137458333.333333,144448275.862069,69888888.888889,2.273438
...,...,...,...,...,...,...,...,...,...,...,...
4381,휴니드테크놀러지스,2024,350.000000,9.000000,26354828000.000000,23753408000.000000,2601420000.000000,73411777.158774,76871870.550162,52028400.000000,8.451532
4382,휴비스,2024,646.000000,58.000000,55799000000.000000,49968000000.000000,5831000000.000000,79259943.181818,80593548.387097,69416666.666667,15.529545
4383,휴스틸,2024,563.000000,22.000000,35118000000.000000,33171000000.000000,1947000000.000000,60030769.230769,61314232.902033,44250000.000000,4.225641
4384,흥국화재,2024,864.000000,97.000000,77082382000.000000,52365382000.000000,24717000000.000000,80210595.213319,103899567.460317,54085339.168490,9.524454


In [8]:
tdf = sdf.merge(wdf, how = 'left', on = ['corp_name', 'year'])
tdf

,corp_code,corp_name,year,SGAE,full_time,part_time,salary_total,salary_total_man,salary_total_woman,salary_ave,salary_ave_man,salary_ave_woman,ave_work
0,365387,AJ네트웍스,2019,NaN,513.000000,18.000000,30112992000.000000,23306579000.000000,6806413000.000000,56709966.101695,67555301.449275,36593618.279570,3.470998
1,125080,AK홀딩스,2019,795724941961.000000,19.000000,1.000000,1302000000.000000,1103000000.000000,199000000.000000,65100000.000000,68937500.000000,49750000.000000,2.300000
2,219097,BGF,2019,101050879822.000000,107.000000,6.000000,8246000000.000000,6864000000.000000,1382000000.000000,72973451.327434,82698795.180723,46066666.666667,6.938053
3,1263022,BGF리테일,2019,859796389576.000000,2010.000000,631.000000,113452000000.000000,92536000000.000000,20916000000.000000,42957970.465733,47723568.849923,29794871.794872,1.900000
4,858364,BNK금융지주,2019,NaN,79.000000,17.000000,13196000000.000000,12567000000.000000,629000000.000000,137458333.333333,144448275.862069,69888888.888889,2.273438
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4381,111421,휴니드테크놀러지스,2024,27557250972.000000,350.000000,9.000000,26354828000.000000,23753408000.000000,2601420000.000000,73411777.158774,76871870.550162,52028400.000000,8.451532
4382,362238,휴비스,2024,84504067956.000000,646.000000,58.000000,55799000000.000000,49968000000.000000,5831000000.000000,79259943.181818,80593548.387097,69416666.666667,15.529545
4383,156488,휴스틸,2024,39800474989.000000,563.000000,22.000000,35118000000.000000,33171000000.000000,1947000000.000000,60030769.230769,61314232.902033,44250000.000000,4.225641
4384,103176,흥국화재,2024,NaN,864.000000,97.000000,77082382000.000000,52365382000.000000,24717000000.000000,80210595.213319,103899567.460317,54085339.168490,9.524454


In [10]:
# 조건: 판관비가 있고, (판관비 - 총급여)가 양수일 때만 계산
# 그렇지 않으면(음수이거나 데이터 없음) -> np.nan (결측치) 처리
tdf['SGAE_etc'] = np.where(
    (tdf['SGAE'].notnull()) & ((tdf['SGAE'] - tdf['salary_total']) > 0), 
    tdf['SGAE'] - tdf['salary_total'], 
    np.nan
)
tdf

,corp_code,corp_name,year,SGAE,full_time,part_time,salary_total,salary_total_man,salary_total_woman,salary_ave,salary_ave_man,salary_ave_woman,ave_work,SGAE_etc
0,365387,AJ네트웍스,2019,NaN,513.000000,18.000000,30112992000.000000,23306579000.000000,6806413000.000000,56709966.101695,67555301.449275,36593618.279570,3.470998,NaN
1,125080,AK홀딩스,2019,795724941961.000000,19.000000,1.000000,1302000000.000000,1103000000.000000,199000000.000000,65100000.000000,68937500.000000,49750000.000000,2.300000,794422941961.000000
2,219097,BGF,2019,101050879822.000000,107.000000,6.000000,8246000000.000000,6864000000.000000,1382000000.000000,72973451.327434,82698795.180723,46066666.666667,6.938053,92804879822.000000
3,1263022,BGF리테일,2019,859796389576.000000,2010.000000,631.000000,113452000000.000000,92536000000.000000,20916000000.000000,42957970.465733,47723568.849923,29794871.794872,1.900000,746344389576.000000
4,858364,BNK금융지주,2019,NaN,79.000000,17.000000,13196000000.000000,12567000000.000000,629000000.000000,137458333.333333,144448275.862069,69888888.888889,2.273438,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4381,111421,휴니드테크놀러지스,2024,27557250972.000000,350.000000,9.000000,26354828000.000000,23753408000.000000,2601420000.000000,73411777.158774,76871870.550162,52028400.000000,8.451532,1202422972.000000
4382,362238,휴비스,2024,84504067956.000000,646.000000,58.000000,55799000000.000000,49968000000.000000,5831000000.000000,79259943.181818,80593548.387097,69416666.666667,15.529545,28705067956.000000
4383,156488,휴스틸,2024,39800474989.000000,563.000000,22.000000,35118000000.000000,33171000000.000000,1947000000.000000,60030769.230769,61314232.902033,44250000.000000,4.225641,4682474989.000000
4384,103176,흥국화재,2024,NaN,864.000000,97.000000,77082382000.000000,52365382000.000000,24717000000.000000,80210595.213319,103899567.460317,54085339.168490,9.524454,NaN


In [11]:
tdf.isnull().sum()

corp_code                0
corp_name                0
year                     0
SGAE                   367
full_time                6
part_time                6
salary_total             6
salary_total_man         6
salary_total_woman       6
salary_ave               6
salary_ave_man           6
salary_ave_woman         6
ave_work                 6
SGAE_etc              1188
dtype: int64

In [14]:
# 1. 일단 정석대로 계산 (음수는 그대로 둠)
tdf['temp_diff'] = tdf['SGAE'] - tdf['salary_total']

# 2. 양수인 기업들의 '판관비 대비 기타비용 비율' 평균 계산
# (제대로 계산된 기업들은 판관비 중 몇 %가 기타비용인지 확인)
valid_ratio = (tdf.loc[tdf['temp_diff'] > 0, 'temp_diff'] / tdf.loc[tdf['temp_diff'] > 0, 'SGAE']).mean()

# 만약 평균 비율이 0.5(50%)라고 나왔다면, 
# 음수가 나온 기업들은 강제로 판관비의 50%를 기타비용으로 추정

# 3. 최종 적용
tdf['SGAE_etc'] = np.where(
    tdf['temp_diff'] > 0,       # 계산이 잘 된 경우
    tdf['temp_diff'],           # 그 값 그대로 사용
    tdf['SGAE'] * valid_ratio   # 계산 안 되면(음수면) 추정 비율 적용
)

# 임시 컬럼 삭제
tdf = tdf.drop(columns=['temp_diff'])

In [15]:
tdf

,corp_code,corp_name,year,SGAE,full_time,part_time,salary_total,salary_total_man,salary_total_woman,salary_ave,salary_ave_man,salary_ave_woman,ave_work,SGAE_etc
0,365387,AJ네트웍스,2019,NaN,513.000000,18.000000,30112992000.000000,23306579000.000000,6806413000.000000,56709966.101695,67555301.449275,36593618.279570,3.470998,NaN
1,125080,AK홀딩스,2019,795724941961.000000,19.000000,1.000000,1302000000.000000,1103000000.000000,199000000.000000,65100000.000000,68937500.000000,49750000.000000,2.300000,794422941961.000000
2,219097,BGF,2019,101050879822.000000,107.000000,6.000000,8246000000.000000,6864000000.000000,1382000000.000000,72973451.327434,82698795.180723,46066666.666667,6.938053,92804879822.000000
3,1263022,BGF리테일,2019,859796389576.000000,2010.000000,631.000000,113452000000.000000,92536000000.000000,20916000000.000000,42957970.465733,47723568.849923,29794871.794872,1.900000,746344389576.000000
4,858364,BNK금융지주,2019,NaN,79.000000,17.000000,13196000000.000000,12567000000.000000,629000000.000000,137458333.333333,144448275.862069,69888888.888889,2.273438,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4381,111421,휴니드테크놀러지스,2024,27557250972.000000,350.000000,9.000000,26354828000.000000,23753408000.000000,2601420000.000000,73411777.158774,76871870.550162,52028400.000000,8.451532,1202422972.000000
4382,362238,휴비스,2024,84504067956.000000,646.000000,58.000000,55799000000.000000,49968000000.000000,5831000000.000000,79259943.181818,80593548.387097,69416666.666667,15.529545,28705067956.000000
4383,156488,휴스틸,2024,39800474989.000000,563.000000,22.000000,35118000000.000000,33171000000.000000,1947000000.000000,60030769.230769,61314232.902033,44250000.000000,4.225641,4682474989.000000
4384,103176,흥국화재,2024,NaN,864.000000,97.000000,77082382000.000000,52365382000.000000,24717000000.000000,80210595.213319,103899567.460317,54085339.168490,9.524454,NaN


In [16]:
tdf.isnull().sum()

corp_code               0
corp_name               0
year                    0
SGAE                  367
full_time               6
part_time               6
salary_total            6
salary_total_man        6
salary_total_woman      6
salary_ave              6
salary_ave_man          6
salary_ave_woman        6
ave_work                6
SGAE_etc              367
dtype: int64

In [17]:
import numpy as np

# 1. NaN 유지 및 0으로 나누기 방지
# 조건: (평균급여가 존재하고) AND (0보다 커야 함)
tdf['man'] = np.where(
    (tdf['salary_ave_man'].notnull()) & (tdf['salary_ave_man'] > 0),  # [수정] 괄호를 각각 감싸야 함
    tdf['salary_total_man'] / tdf['salary_ave_man'], 
    np.nan  # [수정] 조건이 안 맞으면(NaN이거나 0이면) 결측치(NaN) 유지
)

# 2. 여성 인원수 계산 (자연스럽게 NaN 전파)
# 남성(man)이 NaN이면, 결과인 여성(woman)도 자동으로 NaN이 됨 -> 데이터 무결성 유지
tdf['woman'] = (tdf['full_time'] + tdf['part_time']) - tdf['man']

In [18]:
tdf.isnull().sum()

corp_code               0
corp_name               0
year                    0
SGAE                  367
full_time               6
part_time               6
salary_total            6
salary_total_man        6
salary_total_woman      6
salary_ave              6
salary_ave_man          6
salary_ave_woman        6
ave_work                6
SGAE_etc              367
man                    23
woman                  23
dtype: int64

In [26]:
# 남성 평균 급여는 0인데, 여성 평균 급여는 있는 곳 찾기
women_only_corp = tdf[
    (tdf['salary_ave_man'] == 0) & 
    (tdf['salary_ave_woman'] == 0) &
    (tdf['salary_total'] == 0)
]

print(f"여성 직원만 있는 것으로 추정되는 기업 수: {len(women_only_corp)}개")
print(women_only_corp[['corp_name', 'full_time', 'part_time', 'salary_ave_man', 'salary_ave_woman']])

여성 직원만 있는 것으로 추정되는 기업 수: 17개
     corp_name   full_time  part_time  salary_ave_man  salary_ave_woman
346       삼익악기   66.000000   0.000000             NaN               NaN
459      우신시스템  258.000000  21.000000 53437657.894737   35860901.960784
524       조광피혁  157.000000   0.000000             NaN               NaN
563        코아스  261.000000  19.000000 44122950.819672   31694444.444444
690     현대글로비스 1303.000000  89.000000             NaN               NaN
831     SGC에너지  175.000000   0.000000 14406060.606061   11400000.000000
1077      삼익악기   62.000000   1.000000             NaN               NaN
1190     우신시스템  247.000000  16.000000 50740648.648649   42512878.048780
1255      조광피혁  147.000000   0.000000             NaN               NaN
1294       코아스  244.000000  24.000000 41381578.947368   28300000.000000
1421    현대글로비스 1357.000000  90.000000             NaN               NaN
1562    SGC에너지  202.000000   3.000000 82648936.170213   42823529.411765
2025       코아스  244.000000  29.0000

In [23]:
# 2. [추가 요청] "모두 0"인 데이터는 NaN으로 강제 변경 (Cleaning)
# ---------------------------------------------------------------

# 조건: 남성평균, 여성평균, 총급여가 싹 다 0인 경우
all_zero_condition = (
    (tdf['salary_ave_man'] == 0) & 
    (tdf['salary_ave_woman'] == 0) & 
    (tdf['salary_total'] == 0)
)

# 변경할 칼럼 리스트 (급여 관련 + 계산된 인원수)
cols_to_nan = [
    'salary_total', 'salary_total_man', 'salary_total_woman',
    'salary_ave', 'salary_ave_man', 'salary_ave_woman',
    'man', 'woman'
]

# 해당 조건(all_zero_condition)에 맞는 행들의 칼럼 값을 NaN으로 변경
tdf.loc[all_zero_condition, cols_to_nan] = np.nan

# ---------------------------------------------------------------
# 3. 결과 확인
print(f"데이터 정제 완료. NaN으로 변경된 기업 수: {all_zero_condition.sum()}개")

데이터 정제 완료. NaN으로 변경된 기업 수: 17개


In [24]:
tdf.isnull().sum()

corp_code               0
corp_name               0
year                    0
SGAE                  367
full_time               6
part_time               6
salary_total           23
salary_total_man       23
salary_total_woman     23
salary_ave             23
salary_ave_man         23
salary_ave_woman       23
ave_work                6
SGAE_etc              367
man                    23
woman                  23
dtype: int64

In [28]:
# 2. [추가 요청] "모두 0"인 데이터는 NaN으로 강제 변경 (Cleaning)
# ---------------------------------------------------------------

# 조건: 남성평균, 여성평균, 총급여가 싹 다 0인 경우
all_zero_condition = (
    (tdf['ave_work'] == 0)
)

# 변경할 칼럼 리스트 (급여 관련 + 계산된 인원수)
cols_to_nan = [
    'ave_work'
]

# 해당 조건(all_zero_condition)에 맞는 행들의 칼럼 값을 NaN으로 변경
tdf.loc[all_zero_condition, cols_to_nan] = np.nan

# ---------------------------------------------------------------
# 3. 결과 확인
print(f"데이터 정제 완료. NaN으로 변경된 기업 수: {all_zero_condition.sum()}개")

데이터 정제 완료. NaN으로 변경된 기업 수: 17개


In [29]:
tdf.isnull().sum()

corp_code               0
corp_name               0
year                    0
SGAE                  367
full_time               6
part_time               6
salary_total           23
salary_total_man       23
salary_total_woman     23
salary_ave             23
salary_ave_man         23
salary_ave_woman       23
ave_work               23
SGAE_etc              367
man                    23
woman                  23
dtype: int64

In [30]:
import numpy as np

# 1. 성별 비율 생성 (남성 비율, 여성 비율)
# 분모: 남성 수 + 여성 수 (데이터 무결성을 위해 새로 합산)
total_gender = tdf['man'] + tdf['woman']

# 여성 비율 (Female Ratio)
tdf['Female_Ratio'] = np.where(
    (total_gender.notnull()) & (total_gender > 0),
    tdf['woman'] / total_gender,
    np.nan
)


# 2. 고용 형태 비율 생성 (정규직 비율, 비정규직 비율)
# 분모: 정규직 + 비정규직
total_emp = tdf['full_time'] + tdf['part_time']

# 정규직 비율 (Regular Ratio)
tdf['Regular_Ratio'] = np.where(
    (total_emp.notnull()) & (total_emp > 0),
    tdf['full_time'] / total_emp,
    np.nan
)

# 3. 결과 확인
print(tdf[['corp_name', 'Female_Ratio', 'Regular_Ratio']].head(10))

  corp_name  Female_Ratio  Regular_Ratio
0    AJ네트웍스      0.350282       0.966102
1     AK홀딩스      0.200000       0.950000
2       BGF      0.265487       0.946903
3    BGF리테일      0.265808       0.761075
4   BNK금융지주      0.093750       0.822917
5       BYC      0.528701       0.921450
6        CJ      0.307692       0.969231
7    CJ CGV      0.544709       0.715620
8    CJ대한통운      0.147216       0.963196
9     CJ씨푸드      0.469697       0.915584


In [31]:
tdf.isnull().sum()

corp_code               0
corp_name               0
year                    0
SGAE                  367
full_time               6
part_time               6
salary_total           23
salary_total_man       23
salary_total_woman     23
salary_ave             23
salary_ave_man         23
salary_ave_woman       23
ave_work               23
SGAE_etc              367
man                    23
woman                  23
Female_Ratio           23
Regular_Ratio           6
dtype: int64

In [38]:
rdf = tdf.drop(['corp_code', 'SGAE', 'full_time', 'part_time', 'salary_total', 'salary_total_man', 'salary_total_woman',
               'man', 'woman'], axis = 1)
rdf

,corp_name,year,salary_ave,salary_ave_man,salary_ave_woman,ave_work,SGAE_etc,Female_Ratio,Regular_Ratio
0,AJ네트웍스,2019,56709966.101695,67555301.449275,36593618.279570,3.470998,NaN,0.350282,0.966102
1,AK홀딩스,2019,65100000.000000,68937500.000000,49750000.000000,2.300000,794422941961.000000,0.200000,0.950000
2,BGF,2019,72973451.327434,82698795.180723,46066666.666667,6.938053,92804879822.000000,0.265487,0.946903
3,BGF리테일,2019,42957970.465733,47723568.849923,29794871.794872,1.900000,746344389576.000000,0.265808,0.761075
4,BNK금융지주,2019,137458333.333333,144448275.862069,69888888.888889,2.273438,NaN,0.093750,0.822917
...,...,...,...,...,...,...,...,...,...
4381,휴니드테크놀러지스,2024,73411777.158774,76871870.550162,52028400.000000,8.451532,1202422972.000000,0.139276,0.974930
4382,휴비스,2024,79259943.181818,80593548.387097,69416666.666667,15.529545,28705067956.000000,0.119318,0.917614
4383,휴스틸,2024,60030769.230769,61314232.902033,44250000.000000,4.225641,4682474989.000000,0.075214,0.962393
4384,흥국화재,2024,80210595.213319,103899567.460317,54085339.168490,9.524454,NaN,0.475546,0.899063


In [39]:
rdf['SA'] = np.log(rdf['salary_ave'])
rdf

,corp_name,year,salary_ave,salary_ave_man,salary_ave_woman,ave_work,SGAE_etc,Female_Ratio,Regular_Ratio,SA
0,AJ네트웍스,2019,56709966.101695,67555301.449275,36593618.279570,3.470998,NaN,0.350282,0.966102,17.853461
1,AK홀딩스,2019,65100000.000000,68937500.000000,49750000.000000,2.300000,794422941961.000000,0.200000,0.950000,17.991435
2,BGF,2019,72973451.327434,82698795.180723,46066666.666667,6.938053,92804879822.000000,0.265487,0.946903,18.105606
3,BGF리테일,2019,42957970.465733,47723568.849923,29794871.794872,1.900000,746344389576.000000,0.265808,0.761075,17.575733
4,BNK금융지주,2019,137458333.333333,144448275.862069,69888888.888889,2.273438,NaN,0.093750,0.822917,18.738831
...,...,...,...,...,...,...,...,...,...,...
4381,휴니드테크놀러지스,2024,73411777.158774,76871870.550162,52028400.000000,8.451532,1202422972.000000,0.139276,0.974930,18.111595
4382,휴비스,2024,79259943.181818,80593548.387097,69416666.666667,15.529545,28705067956.000000,0.119318,0.917614,18.188243
4383,휴스틸,2024,60030769.230769,61314232.902033,44250000.000000,4.225641,4682474989.000000,0.075214,0.962393,17.910368
4384,흥국화재,2024,80210595.213319,103899567.460317,54085339.168490,9.524454,NaN,0.475546,0.899063,18.200166


In [40]:
rdf['SA_M'] = np.log(rdf['salary_ave_man'])
rdf['SA_F'] = np.log(rdf['salary_ave_woman'])
rdf

C:\Users\USER\anaconda3\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,corp_name,year,salary_ave,salary_ave_man,salary_ave_woman,ave_work,SGAE_etc,Female_Ratio,Regular_Ratio,SA,SA_M,SA_F
0,AJ네트웍스,2019,56709966.101695,67555301.449275,36593618.279570,3.470998,NaN,0.350282,0.966102,17.853461,18.028457,17.415384
1,AK홀딩스,2019,65100000.000000,68937500.000000,49750000.000000,2.300000,794422941961.000000,0.200000,0.950000,17.991435,18.048711,17.722521
2,BGF,2019,72973451.327434,82698795.180723,46066666.666667,6.938053,92804879822.000000,0.265487,0.946903,18.105606,18.230716,17.645600
3,BGF리테일,2019,42957970.465733,47723568.849923,29794871.794872,1.900000,746344389576.000000,0.265808,0.761075,17.575733,17.680936,17.209847
4,BNK금융지주,2019,137458333.333333,144448275.862069,69888888.888889,2.273438,NaN,0.093750,0.822917,18.738831,18.788432,18.062417
...,...,...,...,...,...,...,...,...,...,...,...,...
4381,휴니드테크놀러지스,2024,73411777.158774,76871870.550162,52028400.000000,8.451532,1202422972.000000,0.139276,0.974930,18.111595,18.157651,17.767300
4382,휴비스,2024,79259943.181818,80593548.387097,69416666.666667,15.529545,28705067956.000000,0.119318,0.917614,18.188243,18.204929,18.055638
4383,휴스틸,2024,60030769.230769,61314232.902033,44250000.000000,4.225641,4682474989.000000,0.075214,0.962393,17.910368,17.931523,17.605366
4384,흥국화재,2024,80210595.213319,103899567.460317,54085339.168490,9.524454,NaN,0.475546,0.899063,18.200166,18.458935,17.806074


In [41]:
# 2. 성별 임금 격차(Pay Gap) 변수 생성
# 남성 임금이 0이거나 NaN이면 계산 불가 -> NaN 처리
rdf['Pay_Gap'] = np.where(
    (rdf['salary_ave_man'] > 0) & (rdf['salary_ave_woman'] > 0),
    (rdf['salary_ave_man'] - rdf['salary_ave_woman']) / rdf['salary_ave_man'],
    np.nan
)
rdf

,corp_name,year,salary_ave,salary_ave_man,salary_ave_woman,ave_work,SGAE_etc,Female_Ratio,Regular_Ratio,SA,SA_M,SA_F,Pay_Gap
0,AJ네트웍스,2019,56709966.101695,67555301.449275,36593618.279570,3.470998,NaN,0.350282,0.966102,17.853461,18.028457,17.415384,0.458316
1,AK홀딩스,2019,65100000.000000,68937500.000000,49750000.000000,2.300000,794422941961.000000,0.200000,0.950000,17.991435,18.048711,17.722521,0.278332
2,BGF,2019,72973451.327434,82698795.180723,46066666.666667,6.938053,92804879822.000000,0.265487,0.946903,18.105606,18.230716,17.645600,0.442958
3,BGF리테일,2019,42957970.465733,47723568.849923,29794871.794872,1.900000,746344389576.000000,0.265808,0.761075,17.575733,17.680936,17.209847,0.375678
4,BNK금융지주,2019,137458333.333333,144448275.862069,69888888.888889,2.273438,NaN,0.093750,0.822917,18.738831,18.788432,18.062417,0.516167
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4381,휴니드테크놀러지스,2024,73411777.158774,76871870.550162,52028400.000000,8.451532,1202422972.000000,0.139276,0.974930,18.111595,18.157651,17.767300,0.323180
4382,휴비스,2024,79259943.181818,80593548.387097,69416666.666667,15.529545,28705067956.000000,0.119318,0.917614,18.188243,18.204929,18.055638,0.138682
4383,휴스틸,2024,60030769.230769,61314232.902033,44250000.000000,4.225641,4682474989.000000,0.075214,0.962393,17.910368,17.931523,17.605366,0.278308
4384,흥국화재,2024,80210595.213319,103899567.460317,54085339.168490,9.524454,NaN,0.475546,0.899063,18.200166,18.458935,17.806074,0.479446


In [42]:
rdf = rdf.drop(['salary_ave', 'salary_ave_man', 'salary_ave_woman', 'SA_M', 'SA_F'], axis = 1)
rdf

,corp_name,year,ave_work,SGAE_etc,Female_Ratio,Regular_Ratio,SA,Pay_Gap
0,AJ네트웍스,2019,3.470998,NaN,0.350282,0.966102,17.853461,0.458316
1,AK홀딩스,2019,2.300000,794422941961.000000,0.200000,0.950000,17.991435,0.278332
2,BGF,2019,6.938053,92804879822.000000,0.265487,0.946903,18.105606,0.442958
3,BGF리테일,2019,1.900000,746344389576.000000,0.265808,0.761075,17.575733,0.375678
4,BNK금융지주,2019,2.273438,NaN,0.093750,0.822917,18.738831,0.516167
...,...,...,...,...,...,...,...,...
4381,휴니드테크놀러지스,2024,8.451532,1202422972.000000,0.139276,0.974930,18.111595,0.323180
4382,휴비스,2024,15.529545,28705067956.000000,0.119318,0.917614,18.188243,0.138682
4383,휴스틸,2024,4.225641,4682474989.000000,0.075214,0.962393,17.910368,0.278308
4384,흥국화재,2024,9.524454,NaN,0.475546,0.899063,18.200166,0.479446


In [43]:
rdf.isnull().sum()

corp_name          0
year               0
ave_work          23
SGAE_etc         367
Female_Ratio      23
Regular_Ratio      6
SA                23
Pay_Gap           48
dtype: int64

In [45]:
edf_df = pd.DataFrame()
edf_df

""


In [49]:
for year in range(2019, 2025):
    edf = pd.read_csv(f'data/revenue/revenue_{year}.csv')
    edf = edf.replace(0, np.nan)
    edf_df = pd.concat([edf_df, edf])
    
edf_df

,corp_code,corp_name,year,previous_revenue,current_revenue
0,365387,AJ네트웍스,2019,1045526121935.000000,1202793723694.000000
1,125080,AK홀딩스,2019,3711183102751.000000,3759542062510.000000
2,219097,BGF,2019,220593918847.000000,240314527185.000000
3,1263022,BGF리테일,2019,5775862323457.000000,5946068219555.000000
4,858364,BNK금융지주,2019,NaN,NaN
...,...,...,...,...,...
726,111421,휴니드테크놀러지스,2024,228843875014.000000,230800588043.000000
727,362238,휴비스,2024,927679169915.000000,939427591658.000000
728,156488,휴스틸,2024,764841700033.000000,723068271123.000000
729,103176,흥국화재,2024,NaN,NaN


In [51]:
edf_df = edf_df.drop('corp_code', axis = 1)

In [54]:
result_df = rdf.merge(edf_df, how = 'left', on = ['corp_name', 'year'])
result_df

,corp_name,year,ave_work,SGAE_etc,Female_Ratio,Regular_Ratio,SA,Pay_Gap,current_revenue
0,AJ네트웍스,2019,3.470998,NaN,0.350282,0.966102,17.853461,0.458316,1202793723694.000000
1,AK홀딩스,2019,2.300000,794422941961.000000,0.200000,0.950000,17.991435,0.278332,3759542062510.000000
2,BGF,2019,6.938053,92804879822.000000,0.265487,0.946903,18.105606,0.442958,240314527185.000000
3,BGF리테일,2019,1.900000,746344389576.000000,0.265808,0.761075,17.575733,0.375678,5946068219555.000000
4,BNK금융지주,2019,2.273438,NaN,0.093750,0.822917,18.738831,0.516167,NaN
...,...,...,...,...,...,...,...,...,...
4381,휴니드테크놀러지스,2024,8.451532,1202422972.000000,0.139276,0.974930,18.111595,0.323180,230800588043.000000
4382,휴비스,2024,15.529545,28705067956.000000,0.119318,0.917614,18.188243,0.138682,939427591658.000000
4383,휴스틸,2024,4.225641,4682474989.000000,0.075214,0.962393,17.910368,0.278308,723068271123.000000
4384,흥국화재,2024,9.524454,NaN,0.475546,0.899063,18.200166,0.479446,NaN


In [55]:
result_df['SGAE_R'] = (result_df['SGAE_etc'] / result_df['current_revenue']) * 100
result_df

,corp_name,year,ave_work,SGAE_etc,Female_Ratio,Regular_Ratio,SA,Pay_Gap,current_revenue,SGAE_R
0,AJ네트웍스,2019,3.470998,NaN,0.350282,0.966102,17.853461,0.458316,1202793723694.000000,NaN
1,AK홀딩스,2019,2.300000,794422941961.000000,0.200000,0.950000,17.991435,0.278332,3759542062510.000000,21.130843
2,BGF,2019,6.938053,92804879822.000000,0.265487,0.946903,18.105606,0.442958,240314527185.000000,38.618090
3,BGF리테일,2019,1.900000,746344389576.000000,0.265808,0.761075,17.575733,0.375678,5946068219555.000000,12.551898
4,BNK금융지주,2019,2.273438,NaN,0.093750,0.822917,18.738831,0.516167,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4381,휴니드테크놀러지스,2024,8.451532,1202422972.000000,0.139276,0.974930,18.111595,0.323180,230800588043.000000,0.520979
4382,휴비스,2024,15.529545,28705067956.000000,0.119318,0.917614,18.188243,0.138682,939427591658.000000,3.055591
4383,휴스틸,2024,4.225641,4682474989.000000,0.075214,0.962393,17.910368,0.278308,723068271123.000000,0.647584
4384,흥국화재,2024,9.524454,NaN,0.475546,0.899063,18.200166,0.479446,NaN,NaN


In [56]:
result_df[result_df['SGAE_R'] > 100]

,corp_name,year,ave_work,SGAE_etc,Female_Ratio,Regular_Ratio,SA,Pay_Gap,current_revenue,SGAE_R
57,KC그린홀딩스,2019,4.854167,116092223649.000000,0.437500,1.000000,17.800731,0.293014,13683526761.000000,848.408643
788,KC그린홀딩스,2020,5.312500,109776199074.000000,0.437500,1.000000,17.773279,0.222095,14341909861.000000,765.422459
1164,에스케이바이오팜,2020,3.253500,246345840175.000000,0.465000,1.000000,18.268578,0.060664,25999026722.000000,947.519470
1519,KC그린홀딩스,2021,5.328125,105673025675.000000,0.437500,1.000000,17.995395,0.208459,12811421445.000000,824.834513
2175,화승코퍼레이션,2021,12.512903,188711515285.000000,0.096774,0.987097,18.142234,0.321720,77732387464.000000,242.770770
2250,KC그린홀딩스,2022,5.393519,68244354288.000000,0.277778,1.000000,18.133559,0.383729,14342864854.000000,475.806995
2626,에스케이바이오팜,2022,3.753846,316442014616.000000,0.487179,1.000000,18.251330,0.241778,246179120020.000000,128.541370
2631,에이프로젠,2022,5.252833,114368117736.000000,0.484419,0.985836,17.266432,0.287781,78349308044.000000,145.972084
2906,화승코퍼레이션,2022,11.542675,195680557761.000000,0.165605,1.000000,18.019335,0.553950,115516620925.000000,169.396020
4077,아이에이치큐,2024,8.753097,32833289146.000000,0.362832,0.964602,17.907345,0.315890,32431991390.000000,101.237352


In [57]:
result_df = result_df.drop(['SGAE_etc', 'current_revenue'], axis = 1)
result_df

,corp_name,year,ave_work,Female_Ratio,Regular_Ratio,SA,Pay_Gap,SGAE_R
0,AJ네트웍스,2019,3.470998,0.350282,0.966102,17.853461,0.458316,NaN
1,AK홀딩스,2019,2.300000,0.200000,0.950000,17.991435,0.278332,21.130843
2,BGF,2019,6.938053,0.265487,0.946903,18.105606,0.442958,38.618090
3,BGF리테일,2019,1.900000,0.265808,0.761075,17.575733,0.375678,12.551898
4,BNK금융지주,2019,2.273438,0.093750,0.822917,18.738831,0.516167,NaN
...,...,...,...,...,...,...,...,...
4381,휴니드테크놀러지스,2024,8.451532,0.139276,0.974930,18.111595,0.323180,0.520979
4382,휴비스,2024,15.529545,0.119318,0.917614,18.188243,0.138682,3.055591
4383,휴스틸,2024,4.225641,0.075214,0.962393,17.910368,0.278308,0.647584
4384,흥국화재,2024,9.524454,0.475546,0.899063,18.200166,0.479446,NaN


In [58]:
df = pd.read_csv('data/fin/fin_total_all_years.csv')
df

,corp_code,corp_name,year,A_SIZE,LEV,TQ,stock_code,FOR,MSE,ROA,ADV,SGR,R&D,ESG,E,S,G
0,365387,AJ네트웍스,2019,28.220615,80.738103,93.642922,95570,7.950000,50.050000,2.336252,40.874166,1504.195815,NaN,B,D,B,B
1,125080,AK홀딩스,2019,29.096118,66.960476,77.551265,6840,12.750000,64.890000,1.325746,169.563436,130.306046,51.344764,B+,C,B+,B+
2,219097,BGF,2019,28.136260,7.791009,40.132026,27410,9.000000,69.980000,1.056805,NaN,893.977878,NaN,B+,B,A,B+
3,1263022,BGF리테일,2019,28.406859,71.340792,206.196590,282330,34.120000,55.420000,6.968159,3.526700,294.684822,NaN,A,B+,B+,A
4,858364,BNK금융지주,2019,NaN,NaN,NaN,138930,51.700000,9.190000,NaN,NaN,NaN,NaN,A+,B+,A+,A+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4381,111421,휴니드테크놀러지스,2024,26.208110,28.723503,73.880892,5870,24.830000,22.830000,4.457985,22.652109,85.504278,498.391624,D,C,C,D
4382,362238,휴비스,2024,27.363636,69.298348,80.927896,79980,3.380000,51.000000,-17.323999,1.608714,126.643156,42.896749,B+,B+,A,B+
4383,156488,휴스틸,2024,27.989580,21.772717,36.612095,5010,4.200000,48.060000,1.558903,1.330981,-546.170912,4.621818,C,C,C,C
4384,103176,흥국화재,2024,30.187837,94.007928,95.836360,540,2.170000,79.190000,0.827643,NaN,NaN,73.914884,B+,B+,B,B+


In [59]:
cdf = df.merge(result_df, how = 'left', on = ['corp_name', 'year'])
cdf

,corp_code,corp_name,year,A_SIZE,LEV,TQ,stock_code,FOR,MSE,ROA,...,ESG,E,S,G,ave_work,Female_Ratio,Regular_Ratio,SA,Pay_Gap,SGAE_R
0,365387,AJ네트웍스,2019,28.220615,80.738103,93.642922,95570,7.950000,50.050000,2.336252,...,B,D,B,B,3.470998,0.350282,0.966102,17.853461,0.458316,NaN
1,125080,AK홀딩스,2019,29.096118,66.960476,77.551265,6840,12.750000,64.890000,1.325746,...,B+,C,B+,B+,2.300000,0.200000,0.950000,17.991435,0.278332,21.130843
2,219097,BGF,2019,28.136260,7.791009,40.132026,27410,9.000000,69.980000,1.056805,...,B+,B,A,B+,6.938053,0.265487,0.946903,18.105606,0.442958,38.618090
3,1263022,BGF리테일,2019,28.406859,71.340792,206.196590,282330,34.120000,55.420000,6.968159,...,A,B+,B+,A,1.900000,0.265808,0.761075,17.575733,0.375678,12.551898
4,858364,BNK금융지주,2019,NaN,NaN,NaN,138930,51.700000,9.190000,NaN,...,A+,B+,A+,A+,2.273438,0.093750,0.822917,18.738831,0.516167,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4381,111421,휴니드테크놀러지스,2024,26.208110,28.723503,73.880892,5870,24.830000,22.830000,4.457985,...,D,C,C,D,8.451532,0.139276,0.974930,18.111595,0.323180,0.520979
4382,362238,휴비스,2024,27.363636,69.298348,80.927896,79980,3.380000,51.000000,-17.323999,...,B+,B+,A,B+,15.529545,0.119318,0.917614,18.188243,0.138682,3.055591
4383,156488,휴스틸,2024,27.989580,21.772717,36.612095,5010,4.200000,48.060000,1.558903,...,C,C,C,C,4.225641,0.075214,0.962393,17.910368,0.278308,0.647584
4384,103176,흥국화재,2024,30.187837,94.007928,95.836360,540,2.170000,79.190000,0.827643,...,B+,B+,B,B+,9.524454,0.475546,0.899063,18.200166,0.479446,NaN


In [61]:
cdf = cdf.drop(['ADV', 'R&D'], axis = 1)
cdf

,corp_code,corp_name,year,A_SIZE,LEV,TQ,stock_code,FOR,MSE,ROA,...,ESG,E,S,G,ave_work,Female_Ratio,Regular_Ratio,SA,Pay_Gap,SGAE_R
0,365387,AJ네트웍스,2019,28.220615,80.738103,93.642922,95570,7.950000,50.050000,2.336252,...,B,D,B,B,3.470998,0.350282,0.966102,17.853461,0.458316,NaN
1,125080,AK홀딩스,2019,29.096118,66.960476,77.551265,6840,12.750000,64.890000,1.325746,...,B+,C,B+,B+,2.300000,0.200000,0.950000,17.991435,0.278332,21.130843
2,219097,BGF,2019,28.136260,7.791009,40.132026,27410,9.000000,69.980000,1.056805,...,B+,B,A,B+,6.938053,0.265487,0.946903,18.105606,0.442958,38.618090
3,1263022,BGF리테일,2019,28.406859,71.340792,206.196590,282330,34.120000,55.420000,6.968159,...,A,B+,B+,A,1.900000,0.265808,0.761075,17.575733,0.375678,12.551898
4,858364,BNK금융지주,2019,NaN,NaN,NaN,138930,51.700000,9.190000,NaN,...,A+,B+,A+,A+,2.273438,0.093750,0.822917,18.738831,0.516167,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4381,111421,휴니드테크놀러지스,2024,26.208110,28.723503,73.880892,5870,24.830000,22.830000,4.457985,...,D,C,C,D,8.451532,0.139276,0.974930,18.111595,0.323180,0.520979
4382,362238,휴비스,2024,27.363636,69.298348,80.927896,79980,3.380000,51.000000,-17.323999,...,B+,B+,A,B+,15.529545,0.119318,0.917614,18.188243,0.138682,3.055591
4383,156488,휴스틸,2024,27.989580,21.772717,36.612095,5010,4.200000,48.060000,1.558903,...,C,C,C,C,4.225641,0.075214,0.962393,17.910368,0.278308,0.647584
4384,103176,흥국화재,2024,30.187837,94.007928,95.836360,540,2.170000,79.190000,0.827643,...,B+,B+,B,B+,9.524454,0.475546,0.899063,18.200166,0.479446,NaN


In [63]:
idf = pd.read_csv('data/industry_mapping_data.csv')
idf

,stock_code,corp_name,industry
0,95570,AJ네트웍스,산업용 기계 및 장비 임대업
1,6840,AK홀딩스,기타 금융업
2,27410,BGF,기타 금융업
3,282330,BGF리테일,종합 소매업
4,1460,BYC,봉제의복 제조업
...,...,...,...
726,3530,한화투자증권,금융 지원 서비스업
727,1500,현대차증권,금융 지원 서비스업
728,1450,현대해상,보험업
729,540,흥국화재,보험업


In [64]:
end_df = cdf.merge(idf, how = 'left', on = ['corp_name', 'stock_code'])
end_df

,corp_code,corp_name,year,A_SIZE,LEV,TQ,stock_code,FOR,MSE,ROA,...,E,S,G,ave_work,Female_Ratio,Regular_Ratio,SA,Pay_Gap,SGAE_R,industry
0,365387,AJ네트웍스,2019,28.220615,80.738103,93.642922,95570,7.950000,50.050000,2.336252,...,D,B,B,3.470998,0.350282,0.966102,17.853461,0.458316,NaN,산업용 기계 및 장비 임대업
1,125080,AK홀딩스,2019,29.096118,66.960476,77.551265,6840,12.750000,64.890000,1.325746,...,C,B+,B+,2.300000,0.200000,0.950000,17.991435,0.278332,21.130843,기타 금융업
2,219097,BGF,2019,28.136260,7.791009,40.132026,27410,9.000000,69.980000,1.056805,...,B,A,B+,6.938053,0.265487,0.946903,18.105606,0.442958,38.618090,기타 금융업
3,1263022,BGF리테일,2019,28.406859,71.340792,206.196590,282330,34.120000,55.420000,6.968159,...,B+,B+,A,1.900000,0.265808,0.761075,17.575733,0.375678,12.551898,종합 소매업
4,858364,BNK금융지주,2019,NaN,NaN,NaN,138930,51.700000,9.190000,NaN,...,B+,A+,A+,2.273438,0.093750,0.822917,18.738831,0.516167,NaN,기타 금융업
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4381,111421,휴니드테크놀러지스,2024,26.208110,28.723503,73.880892,5870,24.830000,22.830000,4.457985,...,C,C,D,8.451532,0.139276,0.974930,18.111595,0.323180,0.520979,통신 및 방송 장비 제조업
4382,362238,휴비스,2024,27.363636,69.298348,80.927896,79980,3.380000,51.000000,-17.323999,...,B+,A,B+,15.529545,0.119318,0.917614,18.188243,0.138682,3.055591,화학섬유 제조업
4383,156488,휴스틸,2024,27.989580,21.772717,36.612095,5010,4.200000,48.060000,1.558903,...,C,C,C,4.225641,0.075214,0.962393,17.910368,0.278308,0.647584,1차 철강 제조업
4384,103176,흥국화재,2024,30.187837,94.007928,95.836360,540,2.170000,79.190000,0.827643,...,B+,B,B+,9.524454,0.475546,0.899063,18.200166,0.479446,NaN,보험업


In [67]:
# 1. 먼저 컬럼명을 변경합니다 (ave_work -> W_YEAR)
end_df = end_df.rename(columns={'ave_work': 'W_YEAR'})

# 2. 정렬할 컬럼 리스트 정의 (ave_work를 W_YEAR로 수정해서 반영)
# 사용자가 주신 순서 그대로 유지하되, 이름만 바뀐 것으로 적용
new_col_order = [
    'corp_code', 'corp_name', 'stock_code', 'year', 
    'TQ', 'ROA', 'SGR', 'LEV', 'A_SIZE', 'MSE', 'FOR', 
    'W_YEAR', 'Female_Ratio', 'Regular_Ratio', 
    'SA', 'Pay_Gap', 'SGAE_R', 'industry', 
    'ESG', 'E', 'S', 'G'
]

# 3. 컬럼 순서 재정렬 적용
end_df = end_df[new_col_order]

# 결과 확인
print(end_df.columns)
print(end_df.head())

Index(['corp_code', 'corp_name', 'stock_code', 'year', 'TQ', 'ROA', 'SGR',
       'LEV', 'A_SIZE', 'MSE', 'FOR', 'W_YEAR', 'Female_Ratio',
       'Regular_Ratio', 'SA', 'Pay_Gap', 'SGAE_R', 'industry', 'ESG', 'E', 'S',
       'G'],
      dtype='object')
   corp_code corp_name  stock_code  year         TQ      ROA         SGR  \
0     365387    AJ네트웍스       95570  2019  93.642922 2.336252 1504.195815   
1     125080     AK홀딩스        6840  2019  77.551265 1.325746  130.306046   
2     219097       BGF       27410  2019  40.132026 1.056805  893.977878   
3    1263022    BGF리테일      282330  2019 206.196590 6.968159  294.684822   
4     858364   BNK금융지주      138930  2019        NaN      NaN         NaN   

        LEV    A_SIZE       MSE  ...  Female_Ratio  Regular_Ratio        SA  \
0 80.738103 28.220615 50.050000  ...      0.350282       0.966102 17.853461   
1 66.960476 29.096118 64.890000  ...      0.200000       0.950000 17.991435   
2  7.791009 28.136260 69.980000  ...      0.265487  

In [70]:
end_df['Female_Ratio'] = end_df['Female_Ratio'] * 100
end_df['Regular_Ratio'] = end_df['Regular_Ratio'] * 100
end_df['Pay_Gap'] = end_df['Pay_Gap'] * 100
end_df

,corp_code,corp_name,stock_code,year,TQ,ROA,SGR,LEV,A_SIZE,MSE,...,Female_Ratio,Regular_Ratio,SA,Pay_Gap,SGAE_R,industry,ESG,E,S,G
0,365387,AJ네트웍스,95570,2019,93.642922,2.336252,1504.195815,80.738103,28.220615,50.050000,...,35.028249,96.610169,17.853461,45.831611,NaN,산업용 기계 및 장비 임대업,B,D,B,B
1,125080,AK홀딩스,6840,2019,77.551265,1.325746,130.306046,66.960476,29.096118,64.890000,...,20.000000,95.000000,17.991435,27.833182,21.130843,기타 금융업,B+,C,B+,B+
2,219097,BGF,27410,2019,40.132026,1.056805,893.977878,7.791009,28.136260,69.980000,...,26.548673,94.690265,18.105606,44.295843,38.618090,기타 금융업,B+,B,A,B+
3,1263022,BGF리테일,282330,2019,206.196590,6.968159,294.684822,71.340792,28.406859,55.420000,...,26.580841,76.107535,17.575733,37.567805,12.551898,종합 소매업,A,B+,B+,A
4,858364,BNK금융지주,138930,2019,NaN,NaN,NaN,NaN,NaN,9.190000,...,9.375000,82.291667,18.738831,51.616668,NaN,기타 금융업,A+,B+,A+,A+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4381,111421,휴니드테크놀러지스,5870,2024,73.880892,4.457985,85.504278,28.723503,26.208110,22.830000,...,13.927577,97.493036,18.111595,32.318025,0.520979,통신 및 방송 장비 제조업,D,C,C,D
4382,362238,휴비스,79980,2024,80.927896,-17.323999,126.643156,69.298348,27.363636,51.000000,...,11.931818,91.761364,18.188243,13.868209,3.055591,화학섬유 제조업,B+,B+,A,B+
4383,156488,휴스틸,5010,2024,36.612095,1.558903,-546.170912,21.772717,27.989580,48.060000,...,7.521368,96.239316,17.910368,27.830786,0.647584,1차 철강 제조업,C,C,C,C
4384,103176,흥국화재,540,2024,95.836360,0.827643,NaN,94.007928,30.187837,79.190000,...,47.554631,89.906348,18.200166,47.944596,NaN,보험업,B+,B+,B,B+


In [72]:
end_df = end_df.rename(columns={'Female_Ratio': 'Fe_R', 'Regular_Ratio':'Re_R'})

In [73]:
end_df.to_csv('data/end/end_total.csv')